## Feature Engineering
Transform the data to train differents models

In [ ]:
import pandas as pd

### 0. Root repo

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

### 1. Load data

In [ ]:
path_data = 'artifacts/data/data_raw.pkl'
data = pd.read_pickle(path_data)
data.head()

In [ ]:
data.shape

### 2. Delete the region "Total_US" to have only data for each region

In [ ]:
# create a dataframe with all regions, deleting the total_US
regions = [
    "Great_Lakes",
    "Midsouth",
    "Northeast",
    "Northern_New_England",
    "SouthCentral",
    "Southeast",
    "West",
    "Plains",
]
data = data[data.region.isin(regions)]

In [ ]:
data.shape

### 3. Set index date

In [ ]:
data.set_index('date', inplace = True)

In [ ]:
data.head()

### 4. Delete column year and month. Because this feature won't be use.
ONLY IT WILL USE THE COLUMN "peak" to the model, as seasonality feature

In [ ]:
data = data.drop(columns = ['year', 'month'])
data.head()

### Save data to predict basic model. Columns: ["units_sold", "price", "region", "peak"]

In [ ]:
data.head()

In [ ]:
path_data_basic_features = 'artifacts/data/data_basic_features.pkl'
data.to_pickle(path_data_basic_features)

## GENERATE SECOND DATA SET - "prices_regions" - predict demand considering multiple prices

In [ ]:
# filter usefull columns
list_columns_to_comparation = ['units_sold', 'price', 'region', 'peak']
df_filtered_columns = data[list_columns_to_comparation]
df_filtered_columns

In [ ]:
df_filtered_columns.reset_index(inplace = True) # reset index to works codes

In [ ]:
#### Pivot data to generate a pivot table with the prices for each region in one row
df_pivot_prices = df_filtered_columns.pivot(index='date', columns='region', values='price').reset_index()
df_pivot_prices.columns = ['date'] + ['price_' + col.lower().replace(' ', '_') for col in df_pivot_prices.columns[1:]]
df_pivot_prices

In [ ]:
# delete original column price, with the price in this region
df_filtered_columns = df_filtered_columns.drop(columns = ['price']) 
df_filtered_columns.head()

In [ ]:
#### merge the original data with units solds and region with the data that have the price for each region
df_prices_regions = pd.merge(df_filtered_columns, df_pivot_prices, on='date', how='left')
df_prices_regions.head(9)

In [ ]:
#set index date
df_prices_regions.set_index('date', inplace = True)

### Save data to predict demand considering multiple prices - dataset: "prices_regions"
In this example considering prices of all regions

In [ ]:
df_prices_regions.head()

In [ ]:
path_data_prices_regions = 'artifacts/data/data_prices_regions.pkl'
df_prices_regions.to_pickle(path_data_prices_regions)